In [1]:
import cupy as cp
mempool=cp.get_default_memory_pool()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes()) 

0
0


In [2]:
#Preprocessing for neural network using GloVe embedding - speedup spacy
#preprocessing packages
import pandas as pd
import glob
import re
import spacy
import en_core_web_sm

In [3]:
import glob
combined_datalist=[pd.read_csv(i) for i in glob.glob('/home/kysha/occ_combined/*.csv')] 

In [4]:
combined_datalist=pd.concat([pd.DataFrame(combined_datalist[i]) for i in range(len(combined_datalist))], sort=True)

In [5]:
rec_data=combined_datalist[['skill_category','skill_title','job_description','city','state']].fillna('None')

In [6]:
with open ('stopwords.txt', 'r') as f:
    word=f.read().split('\n')

word=word
f.close()

In [7]:
def clean_remove_emails_https(X_data):
    X_t=[]
    for i in rec_data.job_description:
        X_t.append(' '.join([p.replace('\n',' ').replace('>','').replace('<','')\
         .replace('-',' ').lower()  for p in i.split()]))

    #remove emails and http(s) streams - can be removed
    collection=[]
    for i in X_t:
        collection.append(re.findall(r'[A-Za-z0-9_\.@+]{8,30}\.edu|[A-Za-z0-9\._@+]{8,30}\.com|\
                                     [A-Za-z0-9_\.@+]{8,30}\.gov|[A-Za-z0-9_\.@+]{8,30}\.net|\
                                     |http://[a-z0-9_\.@+/=?#]{1,150}|https://[a-z0-9_\.@+/=?#]{1,150}|\
                                     www[a-z0-9_\.@+]{8,150}|www[a-z0-9_\.@+]{8,150}', str(i)))
     
    X_emails__http_rm=[]
    for i,j in zip(X_t,collection):
        X_emails__http_rm.append(' '.join([p for p in i.split() if p not in j]))

    return X_emails__http_rm

In [8]:
X = rec_data.job_description

In [9]:
%%time
X_emails__http_rm_data=clean_remove_emails_https(X)

CPU times: user 34 s, sys: 149 ms, total: 34.1 s
Wall time: 34.1 s


In [10]:
print(mempool.used_bytes())             
print(mempool.total_bytes()) 

0
0




Up to this point no GPU memory used

In [11]:
spacy.require_gpu()
nlp = en_core_web_sm.load(batch_size=80)

In [12]:
print(mempool.used_bytes())             
print(mempool.total_bytes()) 

47202304
48313344


In [13]:
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')
print(nlp.pipe_names)

['tagger']


In [14]:
print(mempool.used_bytes())             
print(mempool.total_bytes()) 

14868480
48313344




After loading Spacy's model, 47,202,304 bytes of GPU memory used.  
To reduce, I dropped the parser and entity extractor since my interest
is only in tagging parts-of-speech.  
Reduced to 14,868,480 bytes with
this modification.


In [15]:
def all_word_pos(data):
    words_with_pos=[]
    for i in nlp.pipe(data):
        words_with_pos.append([[k, k.pos_] for k in i])
    
    all_pos=[]
    for i in words_with_pos:
        wrds=[k[0] for k in i]
        pos=[k[1] for k in i]
        df=pd.concat([pd.Series(wrds, name='words'),pd.Series(pos, name='pos')], axis=1)
        df=df.reset_index().iloc[:,1:]
        all_pos.append(df)
        
    #can subset df after run to focus desired positions
    return all_pos

In [16]:
len(X)

56003



The goal for function all_word_pos is to get 
parts-of-speech for all words for each row of text (56,003 rows) 
using Spacy's processing pipeline rather than base tagger in Spacy
to speed up the processing time; considering a 28 minute total time for 
processing using the base tagger.

The end goal is to process all 56,000 under 10 minutes.

The first iteration below is for the first 3 rows...

In [18]:
%%time 
processed_text=all_word_pos(X_emails__http_rm_data[:3])

CPU times: user 25.9 ms, sys: 16.1 ms, total: 41.9 ms
Wall time: 41 ms


In [19]:
processed_text

[              words    pos
 0                be    AUX
 1                 a    DET
 2            leader   NOUN
 3                 .  PUNCT
 4             teach   VERB
 ..              ...    ...
 563  characteristic   NOUN
 564       protected   VERB
 565              by    ADP
 566             law   NOUN
 567               .  PUNCT
 
 [568 rows x 2 columns],
          words    pos
 0       duties   NOUN
 1      summary   VERB
 2        about    ADP
 3          the    DET
 4     position   NOUN
 ...        ...    ...
 2774         6    NUM
 2775     years   NOUN
 2776       for    ADP
 2777  veterans   NOUN
 2778         .  PUNCT
 
 [2779 rows x 2 columns],
             words    pos
 0               k  PROPN
 1          health  PROPN
 2              is    AUX
 3    transforming   VERB
 4             the    DET
 ..            ...    ...
 497   programming   NOUN
 498           and  CCONJ
 499   distributed   VERB
 500       systems   NOUN
 501        idioms   NOUN
 
 [502 rows x 2 colu

In [20]:
print(mempool.used_bytes())             
print(mempool.total_bytes()) 

2059094016
2753218560


In [16]:
%%time 
processed_text=all_word_pos(X_emails__http_rm_data[:1000])

CPU times: user 9.41 s, sys: 144 ms, total: 9.56 s
Wall time: 9.57 s


In [17]:
print(mempool.used_bytes())             
print(mempool.total_bytes()) 

14868480
2564058624


In [16]:
%%time 
processed_text=all_word_pos(X_emails__http_rm_data[:5000])

OutOfMemoryError: Out of memory allocating 247,775,232 bytes (allocated so far: 2,753,218,560 bytes).



I can successfully run 1000 rows.  
However, the function fails at 5000 rows.  
Below is the memory usage at this point.

In [18]:
print(mempool.used_bytes())             
print(mempool.total_bytes())

1067994112
2560986624
